In [ ]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.utils.data as Data
from matplotlib import pyplot as plt
import numpy as np
import random
from collections import OrderedDict

In [ ]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [ ]:
batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [ ]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.4374,  0.5995],
        [ 0.4122,  0.5882],
        [ 1.8373,  0.9959],
        [ 0.7302,  0.7738],
        [-0.6952,  0.8998],
        [-0.0305, -0.6934],
        [-1.1754, -1.7822],
        [ 0.0493,  0.9525],
        [ 0.1703, -1.4612],
        [-0.5304,  0.0364]]) tensor([ 3.0301,  3.0280,  4.5029,  3.0416, -0.2570,  6.5012,  7.9067,  1.0418,
         9.5110,  3.0162])


In [ ]:
class LinearNet(nn.Module):
    def __init__(self,n_feature):
        super(LinearNet,self).__init__()
        self.linear=nn.Linear(n_feature,1)
    
    def forward(self,x):
        y=self.linear(x)
        return y
    
net=LinearNet(num_inputs)
print(net)
for param in net.parameters():
    print(param)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Parameter containing:
tensor([[-0.2600, -0.1958]], requires_grad=True)
Parameter containing:
tensor([0.2539], requires_grad=True)


可以使用`nn.Sequential`来更加方便地搭建网络，`Sequential`是一个有序的容器，网络层将按照传入`Sequential`的顺序被添加到计算图中。

In [ ]:
net=nn.Sequential(
    nn.Linear(num_inputs,1)
)
net=nn.Sequential(OrderedDict([
    ('linear',nn.Linear(num_inputs,1))
]))
print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [ ]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1234, -0.4897]], requires_grad=True)
Parameter containing:
tensor([0.1589], requires_grad=True)


### 初始化模型参数
将权重参数的每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。

将偏差初始化为0.

In [ ]:
from torch.nn import init

init.normal_(net[0].weight,mean=0,std=0.01)
init.constant_(net[0].bias,val=0)

Parameter containing:
tensor([0.], requires_grad=True)

### 定义损失函数
PyTorch在`nn`模块中提供了各种损失函数，这些这些损失函数可看作是一种特殊的层，PyTorch也将这些损失函数实现为`nn.Module`的子类。我们现在使用它提供的均方误差损失作为模型的损失函数。

In [ ]:
loss=nn.MSELoss()

### 定义优化算法


In [ ]:
import torch.optim as optim

optimizer=optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


### 训练模型

In [ ]:
num_epochs=3
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        output=net(X)
        l=loss(output,y.view(-1,1))
        optimizer.zero_grad()#梯度清零
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000226
epoch 2, loss: 0.000130
epoch 3, loss: 0.000138


In [ ]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 2.0000, -3.4002]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2005], requires_grad=True)
